In [28]:
import argparse
# apply admm matrix factorization -> how to get the first order easily?
from src.preprocess.ml100k import ML100k
from src.models.mf import MF
from src.utils.trainer import Trainer
import torch

argparser = argparse.ArgumentParser()
argparser.add_argument('--latent_dim', type=int, default=10)
argparser.add_argument('--dropout', type=float, default=0.1)
argparser.add_argument('--batch_size', type=int, default=500)
argparser.add_argument('--epochs', type=int, default=100)
argparser.add_argument('--lr', type=float, default=0.01)
argparser.add_argument('--weight_decay', type=float, default=0.1)
args = argparser.parse_args("")
ml100k=ML100k(args)
data=ml100k.get_dataframe()
#mf_model=MF(args,ml100k.num_users,ml100k.num_items)

In [51]:
rating_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [50]:
# train_test_split
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)
import numpy as np

# train the model with Alternating Least Squares

# train the model with Alternating Least Squares
user_embedding=np.random.rand(ml100k.num_users,args.latent_dim)
item_embedding=np.random.rand(ml100k.num_items,args.latent_dim)
MAXITER=20
rating_matrix=data.pivot(index='user',columns='item',values='rating').values
rating_matrix=np.nan_to_num(rating_matrix)
# to integer
rating_matrix=rating_matrix.astype(int)
for iter in range(MAXITER):

    #update user embedding
    for i,ri in enumerate(rating_matrix):
        # user_id=train.iloc[i,0]
        # item_id=train.iloc[i,1]
        # rating=train.iloc[i,2]
        # ie=item_embedding[item_id]
        
        user_embedding[i]=np.linalg.solve(np.dot(item_embedding.T,np.dot(np.diag(ri),item_embedding))+args.weight_decay*np.eye(args.latent_dim),
                                          np.dot(item_embedding.T,np.dot(np.diag(ri),ri.T))).T

    #update item embedding

    for j,rj in enumerate(rating_matrix.T):
        # user_id=train.iloc[i,0]
        item_embedding[j]=np.linalg.solve(np.dot(user_embedding.T,np.dot(np.diag(rj),user_embedding))+args.weight_decay*np.eye(args.latent_dim),
                                            np.dot(user_embedding.T,np.dot(np.diag(rj),rj)))
        #item_embedding[item_id]=np.linalg.solve(np.dot(ue.T,ue)+np.eye(args.weight_decay),np.dot(ue.T,rating))
    
    # printmse
    print("iter: ",iter)
    print("mse: ",np.mean(np.square(rating_matrix-np.dot(user_embedding,item_embedding.T))))

print("done")

iter:  0
mse:  13.168907705357885
iter:  1
mse:  13.14506979136003
iter:  2
mse:  13.001893383667989
iter:  3
mse:  12.845252954929927


KeyboardInterrupt: 